<a href="https://colab.research.google.com/github/TA-aiacademy/course_3.0/blob/CNN/05_CVCNN/Part5_Object_Detection/01new_YOLOv7_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下載課程所需檔案 (YOLOv7, Dataset)

In [ ]:
!wget https://github.com/TA-aiacademy/course_3.0/releases/download/CVCNN_Data/yolo_datasets.zip
!unzip -q yolo_datasets.zip
!wget https://github.com/TA-aiacademy/course_3.0/releases/download/CVCNN_Data/yolov7new.zip
!unzip -q yolov7new.zip

# YOLOv7 實作


## [貓狗公開資料集](https://public.roboflow.com/object-detection/oxford-pets/2/images/fc82071578629d4d44696cb666898d45)
![](https://i.imgur.com/VnNscKi.png)
這個貓狗公開資料集提供了 3680 張影像，為了訓練快一點，這邊只取了 250 張影像來訓練，檔案放在 datasets/pet.zip 中

## 1. 準備資料集
    改變標籤格式
    - 從 Pascal_voc(xml)->Yolo(txt)
    - 從 Coco(json)->Yolo(txt)
![](https://i.imgur.com/eNWUWGQ.png)

* ### Pascal_voc(xml)->Yolo(txt)

In [ ]:
import os
import glob
import random
import shutil
import xml.etree.ElementTree as ET

#讀取資料夾的圖片名稱
def getImagesInDir(dir_path):
    img_formats = ['bmp', 'jpg', 'jpeg', 'png', 'tif', 'tiff', 'dng']
    image_list = []
    for img_format in img_formats:
        for filename in glob.glob(dir_path + f'/*.{img_format}'):
            image_list.append(filename)

    return image_list

# 座標轉換
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x, y, w, h)

# 讀取 annotation 檔案內容並轉換
def convert_annotation(img_path, ann_dir,
                       output_image_path, output_label_path):
    basename = os.path.basename(img_path)
    basename_no_ext = os.path.splitext(basename)[0]

    # copy image
    shutil.copyfile(img_path, os.path.join(output_image_path, basename))

    in_file = open(ann_dir + '/' + basename_no_ext + '.xml')
    out_file = open(output_label_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or difficult == '1':
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
             float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join(
                        [str(a) for a in bb]) + '\n')

In [ ]:
name = 'pet'  # 資料集名稱
classes = ['cat', 'dog']  # 修改自己的類別
train_test_split_rate = 0.2

img_dir = 'datasets/pet_voc/JPEGImages/'  # 照片存放路徑
ann_dir = 'datasets/pet_voc/Annotations/'  # 標籤存放路徑
image_paths = getImagesInDir(img_dir)
random.seed(2022)
random.shuffle(image_paths)

train_image_path = f'datasets/{name}/train/images/'
train_label_path = f'datasets/{name}/train/labels/'
valid_image_path = f'datasets/{name}/valid/images/'
valid_label_path = f'datasets/{name}/valid/labels/'

if not os.path.exists(train_image_path):
    os.makedirs(train_image_path)
if not os.path.exists(train_label_path):
    os.makedirs(train_label_path)
if not os.path.exists(valid_image_path):
    os.makedirs(valid_image_path)
if not os.path.exists(valid_label_path):
    os.makedirs(valid_label_path)

train_test_split = len(image_paths)*train_test_split_rate

for i, img_path in enumerate(image_paths):
    if i >= train_test_split:
        # train
        convert_annotation(img_path, ann_dir,
                           train_image_path, train_label_path)
    else:
        # valid
        convert_annotation(img_path, ann_dir,
                           valid_image_path, valid_label_path)

* ### Coco(json)->Yolo(txt)

In [ ]:
import os
import glob
import random
import json
import shutil

In [ ]:
def getImagesInDir(dir_path):
    img_formats = ['bmp', 'jpg', 'jpeg', 'png', 'tif', 'tiff', 'dng']
    image_list = []
    for img_format in img_formats:
        for filename in glob.glob(dir_path + f'/*.{img_format}'):
            image_list.append(filename)

    return image_list

# 座標轉換
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x, y, w, h)


def convert_annotation(img_path, ann_dir,
                       output_image_path, output_label_path):
    basename = os.path.basename(img_path)
    basename_no_ext = os.path.splitext(basename)[0]

    # copy image
    shutil.copyfile(img_path, os.path.join(output_image_path, basename))

    # get json
    in_file = json.load(open(ann_dir + '/' + basename_no_ext + '.json', encoding="utf-8"))
    out_file = open(output_label_path + basename_no_ext + '.txt', 'w')

    bboxes = []
    labels = []
    for shape in in_file["shapes"]:
        class_name = shape["label"]
        cls_id = class_names.index(class_name)
        (xmin, ymin), (xmax, ymax) = shape["points"]
        xmin, xmax = sorted([xmin, xmax])
        ymin, ymax = sorted([ymin, ymax])
        b = (float(xmin), float(xmax), float(ymin), float(ymax))
        w = int(in_file["imageWidth"])
        h = int(in_file["imageHeight"])
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join(
                        [str(a) for a in bb]) + '\n')

In [ ]:
name = 'pet'  # 資料集名稱
class_names = ['cat', 'dog']  # 修改自己的類別
train_test_split_rate = 0.2

img_dir = 'datasets/pet_coco/'  # 照片存放路徑
ann_dir = 'datasets/pet_coco/'  # 標籤存放路徑
image_paths = getImagesInDir(img_dir)
random.seed(2022)
random.shuffle(image_paths)

train_image_path = f'datasets/{name}/train/images/'
train_label_path = f'datasets/{name}/train/labels/'
valid_image_path = f'datasets/{name}/valid/images/'
valid_label_path = f'datasets/{name}/valid/labels/'

if not os.path.exists(train_image_path):
    os.makedirs(train_image_path)
if not os.path.exists(train_label_path):
    os.makedirs(train_label_path)
if not os.path.exists(valid_image_path):
    os.makedirs(valid_image_path)
if not os.path.exists(valid_label_path):
    os.makedirs(valid_label_path)

train_test_split = len(image_paths)*train_test_split_rate


for i, img_path in enumerate(image_paths):
    if i >= train_test_split:
        # train
        convert_annotation(img_path, ann_dir,
                           train_image_path, train_label_path)
    else:
        # valid
        convert_annotation(img_path, ann_dir,
                           valid_image_path, valid_label_path)

---

## 2. 更改設定檔案
- 修改 cfg/training/yolov7.yaml
- 修改 data/coco.yaml 製作一個自己資料集的 yaml

將yolov7.yaml 設定檔複製一份

!cp 要複製的檔案 新檔案名稱

In [ ]:
!cp cfg/training/yolov7.yaml cfg/training/yolov7-pet.yaml

將class的地方改成自己的class數量

!sed -n -e (顯示) 第幾行 檔案名稱

In [ ]:
!sed -n -e 2p cfg/training/yolov7-pet.yaml



!sed -i (修改) 第幾行/欲修改的字/目標字/ 檔案名稱

In [ ]:
!sed -i '2s/80/2/' cfg/training/yolov7-pet.yaml

In [ ]:
!sed -n -e 2p cfg/training/yolov7-pet.yaml

![](https://i.imgur.com/ZmNHP9h.png)

參考data/coco.yaml 製作一個自己資料集的yaml

In [ ]:
text = \
    """
    train: ./datasets/pet/train # 訓練資料夾位置
    val: ./datasets/pet/valid # 驗證資料夾位置

    # number of classes
    nc: 2 # <-需修改乘自己的類別數量

    # class names
    names: [ 'cat','dog' ]
    """

In [ ]:
with open(f'data/{name}.yaml', 'w') as file:
    file.write(text)

![](https://i.imgur.com/DKR925y.png)

下載預訓練權重檔案
https://github.com/WongKinYiu/yolov7

![](https://i.imgur.com/5jsscuE.png)放置於weights/資料夾底下

執行訓練，訓練參數介紹：
- --weights : 預先訓練的權重路徑(weights/yolov7_training.pt)
- --cfg：模型設定檔案路徑(cfg/training/yolov7-pet.yaml)
- --data：資料集設定檔案路徑(data/pet.yaml)
- --device：GPU設定
- --batch-size：一次訓練照片張數
- --epoch： 訓練圈數

其他可調控參數可置train.py中察看

In [ ]:
!python train.py --weights weights/yolov7_training.pt --cfg cfg/training/yolov7-pet.yaml --data data/pet.yaml --device 0 --batch-size 16 --epoch 50